In [19]:
%load_ext autoreload
%autoreload 2
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import wandb

# import constants
from constants import *

# Add the parent directory to the system path
sys.path.append("..")

from utils import load_data_multilabel_pre_split , create_dataloaders, initialization_using_word2vec, create_subset_dataloader, get_micro_metrics, create_vocabulary_label_pre_split, create_vocabulary
from HAN_model import HierarchicalAttentionNetwork

from metrics import *
from train import train

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
train_dataloader, valid_dataloader, vocab_size = create_dataloaders()

cache_path: ../cache_vocabulary_label_pik/mimic3-ds-50-HAN_word_vocabulary.pik file_exists: True
load_data.started...
load_data_multilabel_new.data_path: ../datasets/data/train_50_eamc.csv
load_data.ended...
load_data.started...
load_data_multilabel_new.data_path: ../datasets/data/dev_50_eamc.csv
load_data.ended...
shuffled training data


In [4]:
model = HierarchicalAttentionNetwork(vocab_size=vocab_size, embed_size=EMBED_SIZE, hidden_size=HIDDEN_SIZE, num_sentences=NUM_SENTENCES, sentence_length=SENTENCE_LENGTH, num_classes=NUM_CLASSES)

In [5]:
model = initialization_using_word2vec(model)

cache_path: ../cache_vocabulary_label_pik/mimic3-ds-50-HAN_word_vocabulary.pik file_exists: True


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.000005)

### first training script, without initialization.
this training script has one major fault: we forgot to reset gradients at each minibatch! Keeping here for reference

In [18]:
# train one epoch
from tqdm import tqdm

losses = []
for i, (x, y) in tqdm(enumerate(dataloader)):
    y_hat = model(x)
    loss = criterion(y_hat, y)
    loss.backward()
    losses.append(loss.item())
    optimizer.step()
    if i % 10 == 0:
        print('iter', i, 'loss', np.array(losses).mean())
print('done 1 epoch', 'loss:', np.array(losses).mean())

1it [00:04,  4.10s/it]

iter 0 loss 52.44690704345703


11it [00:32,  2.65s/it]

iter 10 loss 36.18838847767223


21it [00:58,  2.61s/it]

...

iter 240 loss 29.84288608978398


251it [12:22,  2.78s/it]

iter 250 loss 30.278502186930986


252it [12:25,  2.96s/it]


RuntimeError: shape '[32, 2500, 100]' is invalid for input of size 500000

In [22]:
# train one epoch
from tqdm import tqdm

losses = []
for i, (x, y) in tqdm(enumerate(dataloader)):
    optimizer.zero_grad()
    y_hat = model(x)
    loss = criterion(y_hat, y)
    loss.backward()
    losses.append(loss.item())
    optimizer.step()
    if i % 10 == 0:
        print('iter', i, 'loss', np.array(losses).mean())
print('done 1 epoch', 'loss:', np.array(losses).mean())

1it [00:04,  4.81s/it]

iter 0 loss 75.09092712402344


11it [00:37,  3.11s/it]

iter 10 loss 49.521841916170985


21it [01:04,  2.70s/it]

...

iter 240 loss 31.55847540732736


251it [11:20,  2.64s/it]

iter 250 loss 31.937302692002984


253it [11:23,  2.70s/it]

done 1 epoch loss: 32.009809041682914


Let's evaluate the model after 1 epoch

In [61]:
preds = []
actuals = []
with torch.no_grad():
    for x, y in tqdm(valid_dataloader):
        y_hat = model(x)
        probs = F.softmax(y_hat, dim=1)

        label = torch.argmax(probs, dim=1)
        label_y = torch.argmax(y, dim=1)

        preds.append(list(label.numpy()))
        actuals.append(list(label_y.numpy()))

        

100%|██████████| 50/50 [00:46<00:00,  1.06it/s]


In [63]:
preds = np.concatenate(preds)

In [64]:
actuals = np.concatenate(actuals)

In [65]:
acc, prec, rec, f1 = all_macro(preds, actuals)
acc, prec, rec, f1

(0.3801787164906271,
 0.057303783519039336,
 0.07258064516128919,
 0.06404379062606823)

In [66]:
precision, recall = calculate_precision_recall(preds, actuals)
print(precision, recall)

0.4259376986649714 0.7745664739884393


In [70]:
from sklearn.metrics import precision_score, recall_score

# Assume y_true and y_pred are your actual and predicted labels respectively
# y_true = ...
# y_pred = ...

precision = precision_score(preds, actuals, average='macro')  # for multi-class classification
recall = recall_score(preds, actuals, average='macro')  # for multi-class classification

print(f"Precision: {precision}")
print(f"Recall: {recall}")

Precision: 0.024237049531897863
Recall: 0.01719362244931067


/Users/hungryfoolish/anaconda3/envs/bd4h/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hungryfoolish/anaconda3/envs/bd4h/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [71]:
precision = precision_score(preds, actuals, average='micro')  # for multi-class classification
recall = recall_score(preds, actuals, average='micro')  # for multi-class classification

print(f"Precision: {precision}")
print(f"Recall: {recall}")

Precision: 0.22822631913541006
Recall: 0.22822631913541006


### Training one epoch with smart initialization

In [8]:
# train one epoch
from tqdm import tqdm

losses = []
for i, (x, y) in tqdm(enumerate(train_dataloader)):
    optimizer.zero_grad()
    y_hat = model(x)
    loss = criterion(y_hat, y)
    loss.backward()
    losses.append(loss.item())
    optimizer.step()
    if i % 10 == 0:
        print('iter', i, 'loss', np.array(losses).mean())
print('done 1 epoch', 'loss:', np.array(losses).mean())

1it [00:05,  5.59s/it]

iter 0 loss 22.66522979736328


11it [00:37,  3.26s/it]

iter 10 loss 17.240951017899945


21it [01:05,  2.71s/it]

...

iter 240 loss 23.66225935908274


251it [12:01,  2.90s/it]

iter 250 loss 24.02995038887419


253it [12:04,  2.86s/it]

done 1 epoch loss: 24.120068244783305


loss starts out much lower (less than 40% of the model without initialization)
However loss didn't reduce after 1 epoch. To look into:
- gradient clipping
- learning rate scheduler

In [11]:
logger = SummaryWriter("./logs/")

In [12]:
# train one epoch
from tqdm import tqdm

losses = []
for i, (x, y) in tqdm(enumerate(valid_dataloader)):
    optimizer.zero_grad()
    y_hat = model(x)
    loss = criterion(y_hat, y)
    loss.backward()
    losses.append(loss.item())
    optimizer.step()
    if i % 10 == 0:
        print('iter', i, 'loss', np.array(losses).mean())

        for tag, value in model.named_parameters():
            if value.grad is not None:
                logger.add_histogram(tag + "/grad", value.grad.cpu(), i*10)

print('done 1 epoch', 'loss:', np.array(losses).mean())

0it [00:00, ?it/s]

iter 0 loss 18.12038803100586


10it [00:37,  3.51s/it]

iter 10 loss 19.25037332014604


20it [01:07,  3.55s/it]

...

iter 30 loss 21.38740570314469


40it [02:17,  3.30s/it]

iter 40 loss 22.757802498049852


50it [02:46,  3.34s/it]

done 1 epoch loss: 23.77406593322754


In [19]:
model = HierarchicalAttentionNetwork(vocab_size=vocab_size, embed_size=EMBED_SIZE, hidden_size=HIDDEN_SIZE, num_sentences=NUM_SENTENCES, sentence_length=SENTENCE_LENGTH, num_classes=NUM_CLASSES)
model = initialization_using_word2vec(model)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.000005)

wandb.init(
    # set the wandb project where this run will be logged
    project="bd4h-project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.000005,
    "architecture": "HLAN+LE",
    "dataset": "mimic-50",
    "epochs": 1,
    }
)

wandb.watch(model, criterion, log='all', log_freq=1)

logger = SummaryWriter("./logs/")

losses = []
for i, (x, y) in tqdm(enumerate(valid_dataloader)):
    optimizer.zero_grad()
    y_hat = model(x)
    loss = criterion(y_hat, y)
    loss.backward()
    losses.append(loss.item())
    optimizer.step()
    wandb.log({'loss': loss.item()})

    if i % 10 == 0:
        print('iter', i, 'loss', np.array(losses).mean())

    for layer, (tag, value) in enumerate(model.named_parameters()):
        logger.add_histogram(f"grad/{layer}.{tag}", value.grad.cpu(), i)

print('done 1 epoch', 'loss:', np.array(losses).mean())


cache_path: ../cache_vocabulary_label_pik/mimic3-ds-50-HAN_word_vocabulary.pik file_exists: True


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


loss,▁▅▂▁▁▂▂▄▂▄▄▄▃▄▆▃▃▃▅▄▅▄▄▆▅▇▄▆▆▆▅▇▇▆▇█▆▇▆▂
loss,21.62302


0it [00:00, ?it/s]

iter 0 loss 19.893186569213867


...

iter 30 loss 25.426361330093876


40it [02:30,  3.26s/it]

iter 40 loss 26.752272256990757


50it [03:07,  3.74s/it]

done 1 epoch loss: 27.778998069763183


In [23]:
model.embeddings.weight.data.mean()

tensor(0.0014)

In [26]:
model.embeddings.weight.data

tensor([[ 6.1396e-03,  1.3556e-03, -3.6092e-04,  ..., -5.0547e-03,
         -3.1368e-03,  2.8457e-03],
        [ 2.0926e+00, -4.5464e-01,  1.8291e+00,  ..., -3.2387e+00,
          2.0335e+00,  1.9512e+00],
        [ 1.2300e+00,  1.4408e-01,  4.1557e-01,  ..., -1.0325e+00,
         -3.5222e-01,  1.2765e+00],
        ...,
        [-1.0968e-02, -9.3574e-03,  4.2765e-03,  ..., -9.8647e-03,
         -8.0069e-03, -9.9252e-03],
        [-1.9467e-02, -5.4372e-03,  3.0623e-03,  ...,  9.0115e-02,
         -6.0075e-02, -1.6655e-02],
        [-1.8207e-02,  7.9990e-03,  9.5015e-03,  ...,  3.4038e-02,
         -1.3660e-02, -8.8746e-03]])

In [29]:
model2 = HierarchicalAttentionNetwork(vocab_size=vocab_size, embed_size=EMBED_SIZE, hidden_size=HIDDEN_SIZE, num_sentences=NUM_SENTENCES, sentence_length=SENTENCE_LENGTH, num_classes=NUM_CLASSES)
model2 = initialization_using_word2vec(model2)
model2.embeddings.weight.data

cache_path: ../cache_vocabulary_label_pik/mimic3-ds-50-HAN_word_vocabulary.pik file_exists: True


tensor([[ 7.3915e-04,  4.0171e-03,  5.6544e-03,  ..., -4.6091e-03,
         -6.1196e-03,  8.1953e-04],
        [ 2.0924e+00, -4.5478e-01,  1.8291e+00,  ..., -3.2387e+00,
          2.0336e+00,  1.9510e+00],
        [ 1.2299e+00,  1.4384e-01,  4.1572e-01,  ..., -1.0323e+00,
         -3.5227e-01,  1.2763e+00],
        ...,
        [-1.0968e-02, -9.3574e-03,  4.2765e-03,  ..., -9.8647e-03,
         -8.0069e-03, -9.9252e-03],
        [-1.9467e-02, -5.4372e-03,  3.0623e-03,  ...,  9.0115e-02,
         -6.0075e-02, -1.6655e-02],
        [-1.8207e-02,  7.9990e-03,  9.5015e-03,  ...,  3.4038e-02,
         -1.3660e-02, -8.8746e-03]])

In [30]:
model2.embeddings.weight.data.mean()

tensor(0.0014)

In [35]:
model2.embeddings.weight.data[10,:]

tensor([-3.0873e+00, -1.2834e+00, -3.4314e+00,  2.7273e+00, -1.1918e+00,
        -1.5990e-01, -2.5308e+00,  4.2257e-01, -3.1923e-01,  2.4978e+00,
        -1.2897e-01,  1.7628e+00, -2.2354e-02,  4.0559e+00,  2.1190e-02,
         8.5813e-01,  8.9449e-01,  2.1671e-01,  4.0218e-01, -2.7727e-01,
         1.5968e+00, -1.1943e+00, -1.3494e+00,  2.7491e+00,  1.9064e+00,
        -4.4494e-01,  1.2601e-01,  1.3564e+00, -1.1144e+00,  5.0334e-01,
         1.3037e-01,  9.4918e-01,  3.2484e+00, -5.2825e-01, -6.0826e-03,
         1.9672e+00,  6.7840e-01,  1.4207e+00,  3.2294e+00,  1.4503e+00,
         3.3367e+00, -3.9044e-02, -2.7064e+00,  7.6135e-01,  2.6548e+00,
         1.3578e+00, -2.3968e+00,  1.9119e-01,  1.5119e+00, -8.1424e-01,
        -5.1204e-01, -1.4608e+00, -1.4920e+00,  1.6127e+00, -2.6102e+00,
        -1.9838e+00, -1.5403e+00, -2.4108e+00,  2.4294e+00,  5.9733e-01,
         7.6137e-01,  2.5147e+00, -5.3443e-01, -5.3333e-01, -3.2919e+00,
        -2.8149e+00, -2.2459e+00, -2.8667e-01,  4.5

In [36]:
model.embeddings.weight.data[10,:]

tensor([-3.0871e+00, -1.2833e+00, -3.4314e+00,  2.7275e+00, -1.1919e+00,
        -1.5964e-01, -2.5308e+00,  4.2277e-01, -3.1932e-01,  2.4977e+00,
        -1.2909e-01,  1.7629e+00, -2.2438e-02,  4.0560e+00,  2.0990e-02,
         8.5792e-01,  8.9459e-01,  2.1686e-01,  4.0219e-01, -2.7713e-01,
         1.5968e+00, -1.1944e+00, -1.3493e+00,  2.7493e+00,  1.9063e+00,
        -4.4497e-01,  1.2614e-01,  1.3563e+00, -1.1143e+00,  5.0354e-01,
         1.3013e-01,  9.4936e-01,  3.2485e+00, -5.2824e-01, -6.2785e-03,
         1.9672e+00,  6.7821e-01,  1.4209e+00,  3.2293e+00,  1.4501e+00,
         3.3366e+00, -3.9110e-02, -2.7065e+00,  7.6147e-01,  2.6547e+00,
         1.3579e+00, -2.3970e+00,  1.9140e-01,  1.5119e+00, -8.1431e-01,
        -5.1223e-01, -1.4610e+00, -1.4919e+00,  1.6128e+00, -2.6103e+00,
        -1.9840e+00, -1.5402e+00, -2.4107e+00,  2.4292e+00,  5.9744e-01,
         7.6122e-01,  2.5147e+00, -5.3418e-01, -5.3354e-01, -3.2918e+00,
        -2.8148e+00, -2.2456e+00, -2.8683e-01,  4.5

value of embeddings didn't change much after training

In [38]:
x, y = next(iter(train_dataloader))

In [43]:
x.min()

tensor(0)

In [44]:
y

tensor([[0., 0., 0.,  ..., 0., 1., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

In [45]:
y.shape

torch.Size([32, 50])

In [46]:
y.sum(dim=1)

tensor([ 3.,  3.,  1.,  4.,  6.,  7.,  4.,  6., 10.,  4.,  9.,  5.,  6.,  2.,
         4.,  2.,  3.,  3.,  6.,  1.,  2., 13.,  5.,  3., 12.,  2.,  3.,  5.,
         1.,  9.,  3.,  2.])

we used the wrong loss function! Each y is a multi-hot label, so in stead of CrossEntropyLoss we need to use BCEWithLogitsLoss

In [47]:
model = HierarchicalAttentionNetwork(vocab_size=vocab_size, embed_size=EMBED_SIZE, hidden_size=HIDDEN_SIZE, num_sentences=NUM_SENTENCES, sentence_length=SENTENCE_LENGTH, num_classes=NUM_CLASSES)
model = initialization_using_word2vec(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.000005)

wandb.init(
    # set the wandb project where this run will be logged
    project="bd4h-project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.000005,
    "architecture": "HLAN+LE",
    "dataset": "mimic-50",
    "epochs": 1,
    "loss": "BCEWithLogitsLoss"
    }
)

wandb.watch(model, criterion, log='all', log_freq=1)

logger = SummaryWriter("./logs/")

losses = []
for i, (x, y) in tqdm(enumerate(valid_dataloader)):
    optimizer.zero_grad()
    y_hat = model(x)
    loss = criterion(y_hat, y)
    loss.backward()
    losses.append(loss.item())
    optimizer.step()
    wandb.log({'loss': loss.item()})

    if i % 10 == 0:
        print('iter', i, 'loss', np.array(losses).mean())

    for layer, (tag, value) in enumerate(model.named_parameters()):
        logger.add_histogram(f"grad/{layer}.{tag}", value.grad.cpu(), i)

print('done 1 epoch', 'loss:', np.array(losses).mean())


cache_path: ../cache_vocabulary_label_pik/mimic3-ds-50-HAN_word_vocabulary.pik file_exists: True


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


loss,▁▄▁▁▁▂▂▄▂▄▅▄▃▄▇▃▃▃▅▄▅▄▄▆▅▆▄▆▆▆▅▆▇▇▇█▆▇▇▂
loss,21.271


0it [00:00, ?it/s]

iter 0 loss 0.831281840801239


...

iter 30 loss 0.8223150814733198


40it [02:36,  3.83s/it]

iter 40 loss 0.8104217706657038


50it [03:10,  3.81s/it]

done 1 epoch loss: 0.7969258832931518


yay! now loss did go down during training!

# Overfit on a few data points

In [73]:
import torch
from torch.utils.data import DataLoader, Subset

def create_subset_dataloader(original_loader, num_samples=10):
    """
    Create a new DataLoader with a subset of the original data.

    Parameters:
    - original_loader (DataLoader): The original DataLoader.
    - num_samples (int): Number of samples to include in the subset.

    Returns:
    - DataLoader: A new DataLoader with the subset of data.
    """
    # Get the original dataset from the DataLoader
    original_dataset = original_loader.dataset

    # Check if the desired number of samples is not more than the dataset size
    num_samples = min(num_samples, len(original_dataset))

    # Create a subset
    subset_indices = torch.arange(num_samples)
    subset = Subset(original_dataset, subset_indices)

    # Create a new DataLoader with the subset
    subset_loader = DataLoader(subset, batch_size=original_loader.batch_size, shuffle=False)

    return subset_loader

# Example usage
# Assuming original_loader is your existing DataLoader
new_loader = create_subset_dataloader(train_dataloader, num_samples=1)

In [11]:
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from tqdm import tqdm
import wandb

# import constants
from constants import *

# Add the parent directory to the system path
sys.path.append("..")

from utils import create_vocabulary_label_pre_split, create_vocabulary
from utils import load_data_multilabel_pre_split , create_dataloaders, initialization_using_word2vec
from HAN_model import HierarchicalAttentionNetwork

from metrics import *


def train(dataloader, vocab_size, epochs=1, lr=0.0005, log=True):
    model = HierarchicalAttentionNetwork(vocab_size=vocab_size, embed_size=EMBED_SIZE, hidden_size=HIDDEN_SIZE, num_sentences=NUM_SENTENCES, sentence_length=SENTENCE_LENGTH, num_classes=NUM_CLASSES)
    model = initialization_using_word2vec(model)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

    if log:
        wandb.init(
            # set the wandb project where this run will be logged
            project="bd4h-project",
            
            # track hyperparameters and run metadata
            config={
            "learning_rate": lr,
            "architecture": "HLAN+LE",
            "dataset": "mimic-50",
            "epochs": 1,
            "loss": "BCEWithLogitsLoss"
            }
        )
        wandb.watch(model, criterion, log='all', log_freq=1)

    for epoch in tqdm(range(epochs)):
        losses = []
        for x, y in dataloader:
            optimizer.zero_grad()
            y_hat = model(x)
            loss = criterion(y_hat, y)
            loss.backward()
            losses.append(loss.item())
            optimizer.step()
            
            if log:
                wandb.log({'loss': loss.item()})
                for layer, (tag, value) in enumerate(model.named_parameters()):
                    logger.add_histogram(f"grad/{layer}.{tag}", value.grad.cpu(), i)
            
        print('epoch', epoch, 'loss', np.array(losses).mean())
        scheduler.step(np.array(losses).mean())

    return model


In [50]:
from train import train

In [75]:
model = train(new_loader, vocab_size, epochs=100, lr=0.005, log=False)

cache_path: ../cache_vocabulary_label_pik/mimic3-ds-50-HAN_word_vocabulary.pik file_exists: True


  3%|▎         | 3/100 [00:00<00:11,  8.09it/s]

epoch 0 loss 0.8193429708480835
epoch 1 loss 0.4199869632720947
epoch 2 loss 0.25279635190963745


  5%|▌         | 5/100 [00:00<00:09,  9.56it/s]

epoch 3 loss 0.17087604105472565
epoch 4 loss 0.13766346871852875
epoch 5 loss 0.11568380147218704


...

epoch 92 loss 0.0037333103828132153
epoch 93 loss 0.003689634846523404
epoch 94 loss 0.003645260352641344


 98%|█████████▊| 98/100 [00:09<00:00, 10.39it/s]

epoch 95 loss 0.003603600896894932
epoch 96 loss 0.003564758924767375
epoch 97 loss 0.003527594730257988


100%|██████████| 100/100 [00:09<00:00, 10.64it/s]

epoch 98 loss 0.0034902903717011213
epoch 99 loss 0.0034513953141868114


In [82]:
x, y = next(iter(new_loader))
y_hat = model(x)
print(y.argmax(dim=1))
print(y_hat.argmax(dim=1))

tensor([6])
tensor([48])


In [86]:
y_hat

tensor([[-31.5819, -12.2134, -11.6357, -12.3696, -15.3418, -15.8302,   9.2258,
         -22.1400, -25.6424, -18.1387, -11.7790, -13.0992, -11.1329, -13.5590,
         -22.6651, -12.9194,  -8.6934, -12.4655, -18.7481, -21.6440, -11.1930,
         -11.3491, -11.6307,  -9.9779, -10.5721, -13.8085, -11.0767, -12.3561,
         -11.0915, -14.5768, -13.5439, -25.0359, -14.6869, -21.9130,  -9.2226,
         -10.2231,  -9.9000,   1.6860, -26.5688, -15.5023, -16.1322, -23.2063,
         -13.3837, -42.9854, -15.0577, -15.3846, -13.0794, -14.0496,  16.9560,
         -19.7838]], grad_fn=<AddBackward0>)

In [87]:
y

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [91]:
torch.where(torch.sigmoid(y_hat) > 0.01, torch.ones(y_hat.shape), torch.zeros(y_hat.shape))

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

we have successfully overfit one data point!

# overfit 10 data points

In [6]:
ten_loader = create_subset_dataloader(train_dataloader, num_samples=10)

In [13]:
valid_loader = create_subset_dataloader(valid_dataloader, num_samples=10)

In [20]:
model = train(ten_loader, valid_loader, vocab_size, epochs=5, lr=0.005, log=False, verbose=True)

cache_path: ../cache_vocabulary_label_pik/mimic3-ds-50-HAN_word_vocabulary.pik file_exists: True
Created directory to save model checkpoint at ../checkpoints/WfsQP


1it [00:00,  1.04it/s]00:00<?, ?it/s]
1it [00:00,  3.21it/s]
 20%|██        | 1/5 [00:01<00:05,  1.28s/it]

Epoch 0 train loss: 0.8429926633834839
Epoch 0 valid loss: 0.9579273462295532
Epoch 0 valid accuracy: 0.025974025974025976
Epoch 0 valid precision: 0.11764705882352941
Epoch 0 valid recall: 0.03225806451612903
Epoch 0 valid f1: 0.05063291139240506
--------------------------------------------------


1it [00:00,  1.16it/s]
1it [00:00,  3.26it/s]
 40%|████      | 2/5 [00:02<00:03,  1.22s/it]

Epoch 1 train loss: 0.8514472842216492
Epoch 1 valid loss: 0.9401633739471436
Epoch 1 valid accuracy: 0.02631578947368421
Epoch 1 valid precision: 0.125
Epoch 1 valid recall: 0.03225806451612903
Epoch 1 valid f1: 0.05128205128205128
--------------------------------------------------


...

Epoch 2 train loss: 0.7192738056182861
Epoch 2 valid loss: 0.6289535760879517
Epoch 2 valid accuracy: 0.043478260869565216
Epoch 2 valid precision: 0.11764705882352941
Epoch 2 valid recall: 0.06451612903225806
Epoch 2 valid f1: 0.08333333333333333
--------------------------------------------------


1it [00:00,  1.17it/s]
1it [00:00,  3.34it/s]
 80%|████████  | 4/5 [00:04<00:01,  1.18s/it]

Epoch 3 train loss: 0.435390442609787
Epoch 3 valid loss: 0.7766780853271484
Epoch 3 valid accuracy: 0.09482758620689655
Epoch 3 valid precision: 0.16923076923076924
Epoch 3 valid recall: 0.1774193548387097
Epoch 3 valid f1: 0.1732283464566929
--------------------------------------------------


1it [00:00,  1.17it/s]
1it [00:00,  3.21it/s]
100%|██████████| 5/5 [00:05<00:00,  1.19s/it]

Epoch 4 train loss: 0.40000200271606445
Epoch 4 valid loss: 0.7848633527755737
Epoch 4 valid accuracy: 0.0859375
Epoch 4 valid precision: 0.14285714285714285
Epoch 4 valid recall: 0.1774193548387097
Epoch 4 valid f1: 0.15827338129496404
--------------------------------------------------


In [110]:
x, y = next(iter(ten_loader))
y_hat = model(x)
print(y.argmax(dim=1))

tensor([ 6,  0, 14,  8,  0,  0,  0,  2,  6,  8])


In [112]:
y

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       

In [113]:
torch.where(torch.sigmoid(y_hat) > 0.2, torch.ones(y_hat.shape), torch.zeros(y_hat.shape))

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       

# Overfit 2 batches of data

In [115]:
two_batches_loader = create_subset_dataloader(train_dataloader, num_samples=64)

In [120]:
model = train(two_batches_loader, vocab_size, epochs=100, lr=0.005, log=True)

cache_path: ../cache_vocabulary_label_pik/mimic3-ds-50-HAN_word_vocabulary.pik file_exists: True


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


loss,▁▂▁▂▂▂▂▂▁▁▁▂▂▂▂▂█▃▂▂▂▂▂▂▂▁▁▁▁▁▂▂▁▁▁▁▁▂▂▂
loss,0.42043


  1%|          | 1/100 [00:11<18:33, 11.24s/it]

epoch 0 loss 0.6893196403980255


...

epoch 97 loss 0.11468945443630219


 99%|█████████▉| 99/100 [13:01<00:07,  7.62s/it]

epoch 98 loss 0.1144183985888958


100%|██████████| 100/100 [13:07<00:00,  7.88s/it]

epoch 99 loss 0.11418961361050606


# Overfit to step 600

when training with full data, loss seems to blow up at the end of an epoch, near step 250. Let's triple the data size and see if the same happen to smaller dataset

In [8]:
from utils import create_subset_dataloader
dataloader = create_subset_dataloader(train_dataloader, num_samples=64)

In [11]:
model = train(dataloader, vocab_size, epochs=300, lr=0.005, log=True, verbose=False)

cache_path: ../cache_vocabulary_label_pik/mimic3-ds-50-HAN_word_vocabulary.pik file_exists: True


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


2it [00:12,  6.21s/it] [00:00<?, ?it/s]
2it [00:10,  5.44s/it] [00:12<1:01:54, 12.42s/it]
2it [00:09,  4.79s/it] [00:23<57:11, 11.52s/it]  
2it [00:09,  4.97s/it] [00:32<52:39, 10.64s/it]
2it [00:09,  4.75s/it] [00:42<51:06, 10.36s/it]
2it [00:09,  4.81s/it] [00:52<49:25, 10.05s/it]
2it [00:10,  5.34s/it] [01:01<48:33,  9.91s/it]
2it [00:09,  4.77s/it] [01:12<49:38, 10.16s/it]
2it [00:09,  4.58s/it] [01:22<48:29,  9.97s/it]
2it [00:09,  4.67s/it] [01:31<47:06,  9.71s/it]
2it [00:11,  5.52s/it]0 [01:40<46:24,  9.60s/it]
2it [00:09,  4.67s/it]0 [01:51<48:22, 10.04s/it]
2it [00:09,  4.86s/it]0 [02:01<47:11,  9.83s/it]
2it [00:09,  4.81s/it]0 [02:10<46:52,  9.80s/it]
2it [00:10,  5.31s/it]0 [02:20<46:27,  9.75s/it]
2it [00:09,  4.67s/it]0 [02:31<47:33, 10.01s/it]
2it [00:08,  4.03s/it]0 [02:40<46:26,  9.81s/it]
2it [00:06,  3.40s/it]0 [02:48<43:48,  9.29s/it]
2it [00:08,  4.10s/it]0 [02:55<40:07,  8.54s/it]
2it [00:08,  4.18s/it]0 [03:03<39:31,  8.44s/it]
2it [00:06,  3.42s/it]0 [03:11<39:

all look good. weird that full data training has loss going up. Trying again with shuffled data and high lr

# Train for real

In [106]:
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from tqdm import tqdm
import wandb

# import constants
from constants import *

# Add the parent directory to the system path
sys.path.append("..")

from HLAN.data_util_gensim import create_vocabulary_label_pre_split, create_vocabulary
from utils import load_data_multilabel_pre_split , create_dataloaders, initialization_using_word2vec
from HAN_model import HierarchicalAttentionNetwork

from metrics import *


def train(dataloader, vocab_size, epochs=1, lr=0.0005, log=True, project="bd4h-project"):
    model = HierarchicalAttentionNetwork(vocab_size=vocab_size, embed_size=EMBED_SIZE, hidden_size=HIDDEN_SIZE, num_sentences=NUM_SENTENCES, sentence_length=SENTENCE_LENGTH, num_classes=NUM_CLASSES)
    model = initialization_using_word2vec(model)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

    if log:
        wandb.init(
            # set the wandb project where this run will be logged
            project=project,
            
            # track hyperparameters and run metadata
            config={
            "learning_rate": lr,
            "architecture": "HLAN+LE",
            "dataset": "mimic-50",
            "epochs": epochs,
            "loss": "BCEWithLogitsLoss"
            }
        )
        wandb.watch(model, criterion, log='all', log_freq=1)

    logger = SummaryWriter("./logs/")

    for epoch in tqdm(range(epochs)):
        losses = []
        for x, y in dataloader:
            optimizer.zero_grad()
            y_hat = model(x)
            loss = criterion(y_hat, y)
            loss.backward()
            losses.append(loss.item())
            optimizer.step()
            
            if log:
                wandb.log({'loss': loss.item()})
                for layer, (tag, value) in enumerate(model.named_parameters()):
                    logger.add_histogram(f"grad/{layer}.{tag}", value.grad.cpu(), i)
            
        print('epoch', epoch, 'loss', np.array(losses).mean())
        scheduler.step(np.array(losses).mean())

    return model


In [93]:
model = train(train_dataloader, vocab_size, epochs=5, lr=0.00005, log=True)

cache_path: ../cache_vocabulary_label_pik/mimic3-ds-50-HAN_word_vocabulary.pik file_exists: True


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


loss,█▆▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,0.03946


 20%|██        | 1/5 [13:39<54:37, 819.47s/it]

epoch 0 loss 0.38444908982209064


...

epoch 2 loss 0.3494505654917404


 80%|████████  | 4/5 [53:56<13:31, 811.80s/it]

epoch 3 loss 0.34063784954811743


100%|██████████| 5/5 [1:07:04<00:00, 804.86s/it]

epoch 4 loss 0.3346067282404353


In [94]:
x, y = next(iter(new_loader))

In [95]:
y

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [100]:
y_hat = model(x)
torch.where(torch.sigmoid(y_hat) > 0.1, torch.ones(y_hat.shape), torch.zeros(y_hat.shape))

tensor([[1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1.,
         1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0.]])

In [101]:
x

tensor([[41, 43, 22,  ...,  0,  0,  0]])

In [104]:
x[0,-30:]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])

#### train with higher initial learning rate

In [114]:
model = train(train_dataloader, vocab_size, epochs=5, lr=0.005, log=True)

cache_path: ../cache_vocabulary_label_pik/mimic3-ds-50-HAN_word_vocabulary.pik file_exists: True


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


loss,█▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,0.05901


 20%|██        | 1/5 [13:35<54:23, 815.78s/it]

epoch 0 loss 0.3926349367077643


...

epoch 2 loss 0.4673087099795285


 80%|████████  | 4/5 [55:09<13:53, 833.64s/it]

epoch 3 loss 0.3527893528283349


100%|██████████| 5/5 [1:09:44<00:00, 836.96s/it]

epoch 4 loss 0.34148601904923737


# Train full data 20 epochs

In [14]:
model = train(train_dataloader, vocab_size, epochs=20, lr=0.005, log=True, verbose=False)

cache_path: ../cache_vocabulary_label_pik/mimic3-ds-50-HAN_word_vocabulary.pik file_exists: True


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


loss,█▅▄▃▂▂▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,0.00617


253it [50:00, 11.86s/it]0:00<?, ?it/s]
253it [1:03:29, 15.06s/it]00<15:50:04, 3000.24s/it]
253it [1:10:56, 16.83s/it]3:29<17:22:50, 3476.12s/it]
253it [16:57,  4.02s/it]:04:26<18:05:54, 3832.62s/it]
253it [19:17,  4.57s/it]:21:24<12:05:42, 2721.41s/it]
253it [19:14,  4.56s/it]:40:41<8:59:20, 2157.37s/it] 
253it [21:49,  5.17s/it]:59:56<7:03:50, 1816.45s/it]
253it [1:21:40, 19.37s/it]1:45<5:57:38, 1650.64s/it]
253it [1:23:24, 19.78s/it]3:25<8:57:01, 2685.13s/it]
253it [56:57, 13.51s/it]:06:49<10:25:11, 3410.13s/it]
253it [35:01,  8.30s/it]8:03:47<9:28:43, 3412.40s/it]
253it [14:21,  3.41s/it]8:38:48<7:31:39, 3011.05s/it]
253it [15:58,  3.79s/it]8:53:10<5:14:17, 2357.23s/it]
253it [15:04,  3.57s/it]9:09:08<3:45:35, 1933.59s/it]
253it [15:30,  3.68s/it]9:24:13<2:42:15, 1622.66s/it]
253it [15:14,  3.61s/it]9:39:43<1:57:49, 1413.88s/it]
253it [18:51,  4.47s/it]9:54:57<1:24:13, 1263.47s/it]
253it [14:41,  3.48s/it]10:13:48<1:01:11, 1223.74s/it]
253it [15:34,  3.69s/it]10:28:30<37:21, 1120.93

In [15]:
torch.save(model.state_dict(), "./checkpoints/HLAN_LE_20epochs.pt")

let's evaluate the model. Here we need to use sigmoid activation because it is multiclass

In [ ]:
model = HierarchicalAttentionNetwork(vocab_size=vocab_size, embed_size=EMBED_SIZE, hidden_size=HIDDEN_SIZE, num_sentences=NUM_SENTENCES, sentence_length=SENTENCE_LENGTH, num_classes=NUM_CLASSES)
model.load_state_dict(torch.load("./checkpoints/HLAN_LE_20epochs.pt"))
model.eval()

In [52]:
all_logits = []
all_probs = []
all_labels = []
all_labels_actuals = []
with torch.no_grad():
    for x, y in tqdm(valid_dataloader):
        logits = model(x)
        probs = F.sigmoid(logits)
        labels = torch.where(probs > 0.5, torch.ones(probs.shape), torch.zeros(probs.shape))

        all_logits.append(list(logits.numpy()))
        all_probs.append(list(probs.numpy()))
        all_labels.append(list(labels.numpy()))
        all_labels_actuals.append(list(y.numpy()))

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:55<00:00,  1.11s/it]


In [55]:
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5]
for threshold in thresholds:
    preds = np.concatenate(all_probs)
    actuals = np.concatenate(all_labels_actuals)
    preds = np.where(preds > threshold, 1, 0)
    acc, prec, rec, f1 = all_micro(preds.ravel(), actuals.ravel())
    print(f"Threshold: {threshold}")
    print(f"Accuracy: {acc}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1: {f1}")
    print()

Threshold: 0.1
Accuracy: 0.18715774843641417
Precision: 0.20143605086013464
Recall: 0.725304319724227
F1: 0.3153039243233118

Threshold: 0.2
Accuracy: 0.22248635798800584
Precision: 0.30860311750599523
Recall: 0.44360659269632663
F1: 0.3639899235426703

Threshold: 0.3
Accuracy: 0.21055350553505536
Precision: 0.4007022471910112
Recall: 0.3073359905203059
F1: 0.34786319575687374

Threshold: 0.4
Accuracy: 0.16571075619871173
Precision: 0.47810590631364563
Recall: 0.20230528923839275
F1: 0.284308530769813

Threshold: 0.5
Accuracy: 0.11634076839933183
Precision: 0.5697786333012512
Recall: 0.12754497468490789
F1: 0.20843235630666315



the paper reached 64% F1 after 100 epochs. Ours reached 36% F1 after 30 epochs. Would 70 more epoch help?